<a href="https://colab.research.google.com/github/pandov/cups-mail/blob/master/segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
! sh colab.sh

In [ ]:
%%capture
import torch
from catalyst.utils import get_device, set_global_seed, prepare_cudnn
from catalyst.dl import SupervisedRunner, IouCallback
from src.nn import BACTERIA, DiceLoss, get_segmentation_model
prepare_cudnn(deterministic=True)
set_global_seed(0)
logdir = './logs/segmentation'

In [ ]:
%load_ext tensorboard
%tensorboard --logdir $logdir

In [ ]:
torch.cuda.empty_cache()
model = get_segmentation_model()
model.train()
criterion = DiceLoss()
callbacks = [IouCallback()]
dataset = BACTERIA(target='mask')
loaders = next(dataset.crossval(kfold=4, batch_size=9))
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=40, gamma=0.1)
runner = SupervisedRunner(device=get_device())
runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    callbacks=callbacks,
    loaders=loaders,
    logdir=logdir,
    num_epochs=100,
    # verbose=True,
)

In [ ]:
! zip -r {logdir}.zip {logdir}
! git add logs
! git commit -m 'Changed from Colab'
! git push -u origin master